In [56]:
import pandas as pd

C = pd.read_stata(
    "/Users/clarissaache/Downloads/microdata_2006_07_stata/section c.dta", convert_categoricals=False
)
print(C.shape)
C.sample(5)


(446425, 19)


,hhcode,province,district,region,psu,hhno,idc,sec,scq01,scq02,scq03,scq04,scq05,scq06,scq07,scq08a,scq08b,scq09a,scq09b
24645,1.052003e+09,1,105,2,10520025,13,2,C,1.0,1.0,1,10.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
358835,3.192001e+09,3,319,2,31920005,16,2,C,2.0,2.0,2,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN
357855,3.182002e+09,3,318,2,31820023,5,8,C,NaN,NaN,1,0.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN
173101,1.332000e+09,1,133,2,13320001,10,3,C,1.0,1.0,1,5.0,2.0,NaN,NaN,NaN,NaN,10.0,14.0
143682,1.262001e+09,1,126,2,12620010,15,6,C,1.0,1.0,1,2.0,1.0,3.0,1.0,1.0,NaN,NaN,NaN


In [57]:
# Create a yes/no column for "ever admitted"
C["ever_admitted"] = C["scq03"]
C["max_level_acheived"] = C["scq04"]
C["currently_enrolled"] = C["scq05"] # IMPORTANT!!! (counts of enrrollment)
C["why_not"] = C["scq09a"]

In [58]:
# Reduce to the questions of interest (4 questions):

C_relevant = C[
    [
        "hhcode",
        "idc",
        "ever_admitted",
        "max_level_acheived",
        "currently_enrolled",
        "why_not",
        "region",
        "province",
    ]
].copy()


In [59]:
C_relevant

,hhcode,idc,ever_admitted,max_level_acheived,currently_enrolled,why_not,region,province
0,1.001100e+09,1,2,NaN,NaN,6.0,1,1
1,1.001100e+09,2,2,NaN,NaN,5.0,1,1
2,1.001100e+09,3,2,NaN,NaN,NaN,1,1
3,1.001100e+09,1,1,5.0,2.0,NaN,1,1
4,1.001100e+09,2,2,NaN,NaN,NaN,1,1
...,...,...,...,...,...,...,...,...
446420,4.262002e+09,3,2,NaN,NaN,6.0,2,4
446421,4.262002e+09,4,2,NaN,NaN,4.0,2,4
446422,4.262002e+09,5,2,NaN,NaN,4.0,2,4
446423,4.262002e+09,6,2,NaN,NaN,6.0,2,4


In [60]:
B = pd.read_stata(
    "/Users/clarissaache/Downloads/microdata_2006_07_stata/section b.dta"
)
B.sample(5)

,hhcode,province,district,region,psu,section,idc,sbq01,sbq02,sbq03,sbq41,sbq42,sbq43,age,sbq05
241426,2.072002e+09,sindh,shikarpur,rural,20720024,B,4,female,present,son/daughter,1998,0,0,8,unmarried
318994,2.161302e+09,sindh,karachi,urban,21613018,B,2,female,present,wife/husband,1980,12,14,26,married
144466,1.241101e+09,punjab,multan,urban,12411008,B,1,male,present,head of household,1958,0,0,48,married
301750,2.161106e+09,sindh,karachi,urban,21611061,B,7,male,present,brother/sister,1973,0,0,32,unmarried
333345,3.032000e+09,nwfp,lower dir,rural,30320002,B,9,male,present,nephew/niece,2003,7,22,3,unmarried


In [61]:
# Change the names of the questions for readability
B['sex'] = B['sbq01']
B['marital_status'] = B['sbq05']

B_relevant = B[['hhcode','idc', 'age', 'sex', 'marital_status']].copy()

In [62]:
# Made the Household Code column a str type
B["hhcode"] = B["hhcode"].astype("str")
C["hhcode"] = C["hhcode"].astype("str")

In [63]:
# MERGE
years06_07 = pd.merge(B_relevant, C_relevant, on=["hhcode", "idc"], how='inner', indicator=False)

# Evaluate merge 
#years06_07._merge.value_counts() #(ALL GOOD)

In [64]:
# Select people in school enrollment age (I would say 4-15 years old)
years06_07 = years06_07.loc[(years06_07["age"] >= 4)&(years06_07["age"] <= 15)]

In [65]:
years06_07.province.value_counts()
# weird! It doesnt have the provinces with codes 5,6,7

1    66049
2    42560
3    34792
4    29827
Name: province, dtype: int64

In [66]:
# Lets see which hh codes we have:
years06_07["hhcode"] = years06_07["hhcode"].astype("str")
years06_07['subprovince code']= years06_07['hhcode'].apply(lambda x: x[0:4])
years06_07["subprovince code"] = years06_07["subprovince code"].astype("int")

In [67]:
# Now we import the stratum name - code data
strat_name = pd.read_csv(
    "/Users/clarissaache/Documents/IDS 701/stratum0607_v1.csv"
)
strat_name.sample(5)

,province,subprovince,subprovince code long
49,Sindh,Sukkur,20220
51,Sindh,Nowshero Feroze,20420
4,Punjab,Sialkot,11411
47,Punjab,Rahim Yar Khan,13420
45,Punjab,Bahawalpur,13220


In [68]:
strat_name['subprovince code long']= strat_name['subprovince code long'].astype("str")
strat_name['subprovince code']= strat_name['subprovince code long'].apply(lambda x: x[0:4])
strat_name['subprovince code'] = strat_name['subprovince code'].astype("int")
strat_name.sample(5)

,province,subprovince,subprovince code long,subprovince code
8,Punjab,Bhawalpur,13211,1321
108,Balochistan,MusaKhel,42120,4212
69,NWFP,Malakand,30620,3062
73,NWFP,Nowshera,31020,3102
83,NWFP,Haripur,32020,3202


In [69]:
# MERGE
years06_07_ = pd.merge(years06_07, strat_name, on=["subprovince code"], how='inner', indicator=True)

# Evaluate merge 
years06_07_._merge.value_counts()

both          144401
left_only          0
right_only         0
Name: _merge, dtype: int64

In [70]:
years06_07_.sample(3)

,hhcode,idc,age,sex,marital_status,ever_admitted,max_level_acheived,currently_enrolled,why_not,region,province_x,subprovince code,province_y,subprovince,subprovince code long,_merge
133735,4152000504.0,7,14,male,unmarried,1,3.0,2.0,6.0,2,4,4152,Balochistan,Ketch/Turbat,41520,both
117393,3222001507.0,7,6,male,unmarried,1,0.0,1.0,NaN,2,3,3222,NWFP,LakkiMarwat,32220,both
100528,3072002801.0,4,12,male,unmarried,1,2.0,1.0,NaN,2,3,3072,NWFP,Bonair,30720,both


In [71]:
# add year
years06_07_['year']=2005
years06_07_['marital_status']=years06_07_['marital_status'].astype('str')

#fix marital status categories
years06_07_.loc[(years06_07_['marital_status'] == "unmarried"), 'marital_status'] = 1
years06_07_.loc[(years06_07_['marital_status'] == "married"), 'marital_status'] = 2
years06_07_.loc[(years06_07_['marital_status'] == "only nikha has been solemanised,rukhsati not taken"), 'marital_status'] = 5
years06_07_.loc[(years06_07_['marital_status'] == "widow/widower"), 'marital_status'] = 3
years06_07_.loc[(years06_07_['marital_status'] == "divorced"), 'marital_status'] = 4
years06_07_['marital_status']=years06_07_['marital_status'].astype('int')

# fix column names
years06_07_['province']=years06_07_['province_y']

In [72]:
#select relevant columns
years06_07_ = years06_07_[
    [
        "hhcode",
        "idc",
        "age",
        "marital_status",
        "sex",
        "ever_admitted",
        "currently_enrolled",
        "region",
        "province",
        "subprovince code",
        "province",
        "year",
    ]
].copy()

In [73]:
years06_07_.to_csv('/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/10_data_cleaning/clean_clarissa/years06_07_.csv', index_label="id")